In [1]:
from q_byzantine.agreement import Process, agreement
from q_byzantine.broadcast import broadcasted_messages, first_to_decide
from q_byzantine import shared_state as state
from q_byzantine import broadcast
from protocol_tests import test_all
from q_byzantine.globals import n

import threading
import random
import time

TRIALS = 5
timings = []
avg_epochs = []

for trial in range(TRIALS):
    print(f"\n--- Trial {trial + 1} ---")

    state.processes.clear()
    state.threads.clear()
    broadcasted_messages.clear()

    for i in range(n):
        pr = Process(i, str(random.choice([0, 1])))
        state.processes.append(pr)
        thr = threading.Thread(target=agreement, args=(pr,))
        state.threads.append(thr)

    start = time.time()

    for thr in state.threads:
        thr.start()
    for thr in state.threads:
        thr.join()

    duration = time.time() - start
    timings.append(duration)

    # Log decisions and epochs
    for pr in state.processes:
        print(f"process({pr.id}) = {pr.output} @epoch: {pr.decision_epoch} | input: {pr.input_val} | faulty: {pr.faulty}")

    first = broadcast.first_to_decide
    if first:
        print(f"[First to decide] Process({first}) decided at epoch {pr.decision_epoch}")
    
    epochs = [pr.decision_epoch for pr in state.processes if pr.decision_epoch is not None]
    avg_epoch = sum(epochs) / len(epochs) if epochs else None
    avg_epochs.append(avg_epoch)

    test_all(state.processes, broadcast.first_to_decide, broadcast.broadcasted_messages)

# Final averages
overall_avg_time = sum(timings) / TRIALS
overall_avg_epoch = sum(avg_epochs) / TRIALS

print(f"\n--- Summary over {TRIALS} trials ---")
print(f"Average time per trial: {overall_avg_time:.2f}s")
print(f"Average decision epoch: {overall_avg_epoch:.2f}")



--- Trial 1 ---
[P0] E1 | current=1, decided=False
[P1] E1 | current=0, decided=False
[P2] E1 | current=1, decided=False
[P3] E1 | current=0, decided=False
[P4] E1 | current=1, decided=False
[P5] E1 | current=1, decided=False
[P6] E1 | current=0, decided=False
[P7] E1 | current=1, decided=False
[P8] E1 | current=1, decided=False
[P9] E1 | current=1, decided=False
[P8] E2 | current=1, decided=True
[P7] E2 | current=1, decided=True
[P6] E2 | current=1, decided=True
[P0] E2 | current=1, decided=True
[P3] E2 | current=1, decided=True
[P2] E2 | current=1, decided=True
[P9] E2 | current=1, decided=True
process(0) = 1 @epoch: 1 | input: 1 | faulty: False
process(1) = 1 @epoch: 1 | input: 0 | faulty: True
process(2) = 1 @epoch: 1 | input: 1 | faulty: False
process(3) = 1 @epoch: 1 | input: 0 | faulty: False
process(4) = 1 @epoch: 1 | input: 1 | faulty: True
process(5) = 1 @epoch: 1 | input: 1 | faulty: True
process(6) = 1 @epoch: 1 | input: 0 | faulty: False
process(7) = 1 @epoch: 1 | input: 